In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from scipy import stats
import ipy_table as tbl

run_id = 359
qid = 1603

In [2]:
topics = Topic.objects.filter(run_id=181)
dts = DocTopic.objects.filter(run_id=181)

p1total = dts.filter(doc__PY__gt=2009,doc__PY__lt=2014).aggregate(t=Sum('score'))
p2total = dts.filter(doc__PY__gt=2013,doc__PY__lt=2018).aggregate(t=Sum('score'))

In [3]:
p1total

{'t': 313.828131828615}

In [4]:
p2total

{'t': 482.821877065289}

In [5]:
topic_objects = []
for t in topics:
    t1 = dts.filter(doc__PY__gt=2007,doc__PY__lt=2013,topic=t).aggregate(t=Sum('score'))
    t2 = dts.filter(doc__PY__gt=2012,doc__PY__lt=2018,topic=t).aggregate(t=Sum('score'))
    p1 = t1['t'] / p1total['t']
    p2 = t2['t'] / p2total['t']
    growth = (p2 - p1) / p1*100
    t.growth = float(growth)
    t.save()
    print(t.title)
    print(t.growth)
    t.save()
    topic_objects.append(t)
    
print(topic_objects)
for top in topic_objects:
    print(top.growth)
    print(top.run_id_id)
    top.save()
    
#Topic.objects.update(topics)
t.save()

{bicycl, cycl, cyclist}
38.983610372093956
{park, space, price}
57.71585474271607
{polici, instrument, effect}
18.266387414500056
{urban, land, citi}
47.27189864087317
{transit, rail, station}
30.394877443807328
{vehicl, electr, charg}
76.28782461275621
{recycl, household, collect}
14.044720900164966
{transport, public, access}
9.617688005516303
{water, heat, suppli}
34.004856710783756
{bus, servic, oper}
22.17548574587257
{green, build, roof}
135.42122069972967
{climat, chang, mitig}
18.222567144032283
{heat, thermal, comfort}
32.267698292317974
{toll, lane, link}
-19.341891182904185
{brt, bus, system}
66.36134947819893
{activ, school, health}
13.884107758473075
{air, pollut, qualiti}
8.325408090322556
{model, network, optim}
32.26736430578555
{treatment, wastewat, plant}
-1.9167528998093541
{emiss, carbon, ghg}
29.830327526279188
{travel, mode, choic}
29.09757661019347
{price, congest, road}
-16.58607136873004
{wast, manag, solid}
-19.659099568765043
{energi, build, consumpt}
37.2448

In [6]:
topic_table = topics.values('title','growth')
df = pd.DataFrame.from_dict(list(topic_table))
df.to_csv('../data/topic_growth_2010.csv')